In [1]:
# Import all the necessary library
import pandas as pd
from sqlalchemy import create_engine
from config import username, password

# EXTRACT

In [2]:
# Extract the two dataset CSVs into DataFrames
## Load Happiness data
happiness_file = 'Resources/world_happiness_2019.csv'
happiness_df = pd.read_csv(happiness_file)
happiness_df.head(3)

,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410
2,3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341


In [3]:
countries_file = "Resources/countries of the world.csv"
countries_df = pd.read_csv(countries_file)
countries_df.head(3)

,Country,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,"48,0","0,00","23,06","163,07",700.0,"36,0","3,2","12,13","0,22","87,65",1,"46,6","20,34","0,38","0,24","0,38"
1,Albania,EASTERN EUROPE,3581655,28748,"124,6","1,26","-4,93","21,52",4500.0,"86,5","71,2","21,09","4,42","74,49",3,"15,11","5,22","0,232","0,188","0,579"
2,Algeria,NORTHERN AFRICA,32930091,2381740,"13,8","0,04","-0,39",31,6000.0,"70,0","78,1","3,22","0,25","96,53",1,"17,14","4,61","0,101","0,6","0,298"


# TRANSFORM

Wolrd Countries

In [4]:
## Happiness df ##
# Create a filtered dataframe from specific columns
happiness_df_col = ['Country or region', 'Score', 'GDP per capita',
       'Social support', 'Healthy life expectancy',
       'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption']
happiness_transformed_df = happiness_df[happiness_df_col].copy()

# Rename the column headers
happiness_transformed_df = happiness_transformed_df.rename(columns={
    'Country or region' : 'country',
    'Score': 'overall_score', 
    'GDP per capita': 'gdp_score',
    'Social support': 'social_support', 
    'Healthy life expectancy': 'healthy_life_expectancy', 
    'Freedom to make life choices': 'freedom_choices',
    'Generosity': 'generosity', 
    'Perceptions of corruption': 'corruption_perception'
})

# Clean the data by dropping duplicates and setting the index
happiness_transformed_df = happiness_transformed_df.dropna(how='any')
happiness_transformed_df.drop_duplicates('country', inplace=True)
happiness_transformed_df.set_index('country', inplace=True)
happiness_transformed_df

,overall_score,gdp_score,social_support,healthy_life_expectancy,freedom_choices,generosity,corruption_perception
country,,,,,,,
Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393
Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410
Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341
Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118
Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298
...,...,...,...,...,...,...,...
Rwanda,3.334,0.359,0.711,0.614,0.555,0.217,0.411
Tanzania,3.231,0.476,0.885,0.499,0.417,0.276,0.147
Afghanistan,3.203,0.350,0.517,0.361,0.000,0.158,0.025


Wolrd Happiness

In [5]:
## Countries df ##
# Create a filtered dataframe from specific columns
countries_df_col = ['Country', 'Region', 'Pop. Density (per sq. mi.)', 'Infant mortality (per 1000 births)', 
                    'GDP ($ per capita)', 'Literacy (%)']
countries_transformed_df = countries_df[countries_df_col].copy()

# Rename the column headers
countries_transformed_df = countries_transformed_df.rename(columns={
    'Country': 'country', 
    'Region': 'region', 
    'Pop. Density (per sq. mi.)': 'pop_density', 
    'Infant mortality (per 1000 births)': 'infant_mortality', 
    'GDP ($ per capita)': 'gdp', 
    'Literacy (%)': 'literacy'
})

# Convert columns with string of numbers to float
countries_transformed_df['pop_density'] = countries_transformed_df.pop_density.str.replace(',', '.').astype(float)
countries_transformed_df['infant_mortality'] = countries_transformed_df.infant_mortality.str.replace(',', '.').astype(float)
countries_transformed_df['literacy'] = countries_transformed_df.literacy.str.replace(',', '.').astype(float)

# Remove the weird space after text in country column
countries_transformed_df['country'] = countries_transformed_df['country'].str.strip()

# Clean the data by dropping NaN, duplicates and setting the index
countries_transformed_df = countries_transformed_df.dropna(how='any')
countries_transformed_df.drop_duplicates('country', inplace=True)
countries_transformed_df.set_index('country', inplace=True)
countries_transformed_df

,region,pop_density,infant_mortality,gdp,literacy
country,,,,,
Afghanistan,ASIA (EX. NEAR EAST),48.0,163.07,700.0,36.0
Albania,EASTERN EUROPE,124.6,21.52,4500.0,86.5
Algeria,NORTHERN AFRICA,13.8,31.00,6000.0,70.0
American Samoa,OCEANIA,290.4,9.27,8000.0,97.0
Andorra,WESTERN EUROPE,152.1,4.05,19000.0,100.0
...,...,...,...,...,...
Venezuela,LATIN AMER. & CARIB,28.2,22.20,4800.0,93.4
Vietnam,ASIA (EX. NEAR EAST),256.1,25.95,2500.0,90.3
Yemen,NEAR EAST,40.6,61.50,800.0,50.2


In [11]:
# Create database connection
connection_string = f'postgres:1327@localhost:5432/countries_happiness_db'
engine = create_engine(f'postgresql://{connection_string}')

In [12]:
# Confirm tables
engine.table_names()

['country', 'happiness']

# LOAD

Following the creation of the tables, we loaded the two dataframes into individual databases. 
We then merged the individual dataframes in order to get the final dataframe. 
We chose to combine these two datasets because we feel that the values in both sets could have an impact on the overall happiness score for each country.

In [13]:
# Load Happiness Df into database
happiness_transformed_df.to_sql(name='happiness', con=engine, if_exists='append', index=True)

In [14]:
# Load Countries Df into database
countries_transformed_df.to_sql(name='country', con=engine, if_exists='append', index=True)

### Checking to make sure the two tables merge

In [15]:
# Test merging
merge_df = pd.merge(happiness_transformed_df, countries_transformed_df, on='country')
merge_df

,overall_score,gdp_score,social_support,healthy_life_expectancy,freedom_choices,generosity,corruption_perception,region,pop_density,infant_mortality,gdp,literacy
country,,,,,,,,,,,,
Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393,WESTERN EUROPE,15.5,3.57,27400.0,100.0
Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410,WESTERN EUROPE,126.5,4.56,31100.0,100.0
Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341,WESTERN EUROPE,14.2,3.70,37800.0,100.0
Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118,WESTERN EUROPE,2.9,3.31,30900.0,99.9
Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298,WESTERN EUROPE,397.1,5.04,28600.0,99.0
...,...,...,...,...,...,...,...,...,...,...,...,...
Malawi,3.410,0.191,0.560,0.495,0.443,0.218,0.089,SUB-SAHARAN AFRICA,109.8,103.32,600.0,62.7
Yemen,3.380,0.287,1.163,0.463,0.143,0.108,0.077,NEAR EAST,40.6,61.50,800.0,50.2
Rwanda,3.334,0.359,0.711,0.614,0.555,0.217,0.411,SUB-SAHARAN AFRICA,328.4,91.23,1300.0,70.4
